<a href="https://colab.research.google.com/github/cagBRT/Data/blob/main/3_MissingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Why is data missing?**
Understanding the reasons why data are missing is important for handling the remaining data correctly. If values are missing completely at random, the data sample is likely still representative of the population. But if the values are missing systematically, analysis may be biased. For example, in a study of the relation between IQ and income, if participants with an above-average IQ tend to skip the question ‘What is your salary?’, analyses that do not take into account this missing at random (MAR pattern) may falsely fail to find a positive association between IQ and salary (Wikipedia)

In [ ]:
# Clone the entire repo.
!git clone -s https://github.com/cagBRT/Data.git cloned-repo
%cd cloned-repo

**Methods to handle missing data**<br>
>Deleting rows with missing values<br>
Impute missing values for continuing variable<br>
Impute missing values for categorical variable<br>
Other imputation methods<br>
Using algorithms that support missing values<br>
Prediction of missing values<br>
Imputation using Deep Learning Library - Datawig

In [ ]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

Using the Titanic dataset decide what to do with missing data

In [ ]:
data = pd.read_csv("titanic_train.csv")
data.shape

Find the missing data

In [ ]:
msno.matrix(data)
#White horizontal lines are missing data

There is a lot of missing data in some of the columns. <br>
We need to decide what to do about the missing data

In [ ]:
print(data.isnull().sum())

In [ ]:
data

# **There is a lot of missing data**

If we delete all rows with missing data, our model will be robust. <br>
But if we delete all rows with missing data, we will have a small dataset. 

If we look first at the missing data in the cabins column. We see there is a lot, 687. <br>
The problem we are trying to solve may not require cabin information, so let's delete the entire column. <br>
If the cabin information is required for solving our problem, then we will need to do a little more work. 

**Dropping a column**<br>
For missing cabins, let's drop that column

In [ ]:
df = data.drop(['Cabin'],axis=1)

In [ ]:
msno.matrix(df)

**What about embarked?**<br>
As you can see below, the two rows that are missing the embark port have all the other data. <br>
Depending upon the problem we are trying to solve, we can do different things. 

The embark ports are:<br>
>Southhampton<br>
Cherbourg<br>
Queentown<br>

In [ ]:
check_rows=pd.isnull(df['Embarked'])
df[check_rows]

First let's see if the embark location is important to survival

In [ ]:
tt= df.dropna(subset=['Embarked'])
ports=df['Embarked'].value_counts()
print(ports)
names=tt['Embarked'].unique()
print(names)

survival= df['Survived']==1
portEmbarkS=df['Embarked']=='S'
portEmbarkQ=df['Embarked']=='Q'
portEmbarkC=df['Embarked']=='C'

embarkSurvivedS=df.loc[survival & portEmbarkS]
embarkSurvivedC=df.loc[survival & portEmbarkC]
embarkSurvivedQ=df.loc[survival & portEmbarkQ]
es=[embarkSurvivedS,embarkSurvivedC, embarkSurvivedQ]
esN=[len(embarkSurvivedS),len(embarkSurvivedC), len(embarkSurvivedQ)]

As we can see by the ratios below, where a passenger embarked seems to be important. <br>

In [ ]:
survival_ratio_S=len(embarkSurvivedS)/int(ports[0])
survival_ratio_C=len(embarkSurvivedC)/int(ports[1])
survival_ratio_Q=len(embarkSurvivedQ)/int(ports[2])
print("S:",survival_ratio_S)
print("C:",survival_ratio_C)
print("Q:",survival_ratio_Q)

Same information, in a bar chart

In [ ]:
width=0.2
N=3
ind=np.arange(N)
fig,ax=plt.subplots()
rect1=ax.bar(ind,ports,width)
rect2=ax.bar(ind+width,esN,width)

ax.set_xlabel('Survial')
ax.set_ylabel('Ports')
ax.set_xticks(ind+width/2)
ax.set_xticklabels(('S','Q','C'))
plt.show()

**Imputation for categorical columns**<br>
Instead of deleting the rows, we might decide to insert a value. <br>
For categorical data, it is usual to replace the missing data with the most frequent category.

In [ ]:
df.value_counts(['Embarked'])

Replacing the missing data with the most frequent category may be the right thing to do. <br>
But instead of jumping and doing the usual, let's take a moment and see if we can find any hints to help us decide what to do.

Notice the two missing embark data points are for two women.<br>
They are on the same ticket and the fare is the same for both women.<br>
They are also in the same passenger class, and both survived. <br>
Neither one had siblings or parents on board. <br>
It is possible one is a companion to the other. 

Also notice, their passenger Id numbers are very different. <br>
They did not board the ship at the same time, it is possible they boarded<br>
at very different times. <br>
It is possible the companion boarded in one port and the other woman boarded in a different port. <br>
It is also possible their ticket numbers were entered incorrectly. <br>
But it is an interesting coincidence that their fares and ticket numbers are the same.

In [ ]:
df[check_rows]

Let's look at the data again, and include the sex of the survivors<br>


**Assignment 1:**<BR>
Find the ratio for each number of females who survived from each embark port

So, we have a choice<br>
1. We can delete these two women from the dataset
2. We can assign them the most populous embark port, South Hampton
3. We can assign them the more likely embark port, Cherbourg<br>

Also, with just two missing out of 891 rows, will it be a big impact on the model?

In [ ]:
value = 'C'

df['Embarked'] = df['Embarked'].fillna(value)

Replace the missing values in the embark column

In [ ]:
df

We have removed the cabin column for now. <br>
And we have assigned the two missing embark values with 'C'<br>
Now we have to decide what to do about the missing ages. <br>
Our choices: <bR>
> Delete all the missing ages<br>
Fill with a single value<br>
Fill with other values

In [ ]:
print(df.isnull().sum())

**What is the effect of age on survivability?**<br>
Drop the missing age values and plot age vs survival.<br>


In [ ]:
survived = 'survived'
not_survived = 'not survived'
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10, 4))
women = df[df['Sex']=='female']
men = df[df['Sex']=='male']
#put the ages into bins 
ax = sns.histplot(women[women['Survived']==1].Age.dropna(), bins=18, 
                  label = survived, ax = axes[0], kde =False)
ax = sns.histplot(women[women['Survived']==0].Age.dropna(), bins=18, 
                  label = not_survived, ax = axes[0], kde =False, color='red')
ax.legend()
ax.set_title('Female')
ax = sns.histplot(men[men['Survived']==1].Age.dropna(), bins=18, 
                  label = survived, ax = axes[1], kde = False)
ax = sns.histplot(men[men['Survived']==0].Age.dropna(), bins=18, 
                  label = not_survived, ax = axes[1], kde = False, color='red')
ax.legend()
_ = ax.set_title('Male')

How many people that survived are missing age data?

In [ ]:
df

Notice when we look for passengers that have survived and we do not have an age for them, we see the total is zero. <br>
If we don't know someone's age, then they did not survive. 

In [ ]:
survival= df['Survived']==1
aged=df['Age'].isna()==True
survivedAge=df.loc[survival & aged]
survivedAge

In [ ]:
survival_no= df['Survived']==0
aged=df['Age'].isna()==True
survivedAge_not=df.loc[survival_no & aged]
survivedAge_not

**Assignmen**t <br>
The decision we have to make in this case can be considered an artifical one. <br>
To help our model on Titanic, we might want to set all the missing ages to 0. 
For a production model, we would not want to do this. <br>
**Why?**

**Assignment 2**<br>
>Set the missing ages to 0<br>
>Later we will set the ages to random values and compare the model performance. 

In [ ]:
#@title Assignment 1
#Assignment 1
sex= df.Sex=='female'
embarkSurvivedS=df.loc[survival & portEmbarkS &sex]
embarkSurvivedC=df.loc[survival & portEmbarkC &sex]
embarkSurvivedQ=df.loc[survival & portEmbarkQ &sex]

survival_ratio_S=len(embarkSurvivedS)/int(ports[0])
survival_ratio_C=len(embarkSurvivedC)/int(ports[1])
survival_ratio_Q=len(embarkSurvivedQ)/int(ports[2])
print("S:",survival_ratio_S)
print("C:",survival_ratio_C)
print("Q:",survival_ratio_Q)

In [ ]:
#@title Assignment 2
aged=df['Age'].isna()==True
df[aged]=0
df